In [2]:
import os
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Activation, Dropout, BatchNormalization

!pip install wandb
import wandb
from wandb.keras import WandbCallback
!pip install wandb --upgrade

In [3]:
no_of_filters = [8, 16, 16, 32, 32]
kernel_size =   [3 , 5 , 5 , 5 , 5]
activation  =   tf.nn.relu
dense_layer = 100
image_size = 200
batch_size = 50
epochs = 10

#Dropout and Batch Normalization
dropout=0.0
batch_norm=False

augment_data=False

In [4]:
#TensorFlow - Building the Model
def createCNN(image_size, no_of_filters, kernel_size, activation, batch_norm, dropout, dense_size):

    modeltf = Sequential()

    # layer1
    modeltf.add(Conv2D(input_shape=(image_size, image_size, 3), filters=no_of_filters[0], kernel_size=kernel_size[0], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer2
    modeltf.add(Conv2D(no_of_filters[1], kernel_size=kernel_size[1], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer3
    modeltf.add(Conv2D(no_of_filters[2], kernel_size=kernel_size[2], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer4
    modeltf.add(Conv2D(no_of_filters[3], kernel_size=kernel_size[3], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer5
    modeltf.add(Conv2D(no_of_filters[4], kernel_size=kernel_size[4], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    modeltf.add(Flatten())

    # dense layer
    modeltf.add(Dense(dense_layer, activation=activation))
    modeltf.add(Dropout(dropout))

    # output layer
    modeltf.add(Dense(10, activation=tf.nn.softmax))
    
    return modeltf

In [5]:
# Prepare the dataset for training
def prepare_dataset(DATA_DIR="inaturalist_12K", augment_data=False):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [6]:
#Customise run names for WandB to enhance readability
def setRunName(no_of_filters = [8, 16, 16, 32, 32], augment_data=False, dropout=0.0, batch_norm=False):
    
    augment_data_options = {True: "Y", False: "N"}
    batch_norm_options = {True: "Y", False: "N"}

    run_name = "_".join(["filters", str(no_of_filters), "aug", augment_data_options[augment_data],
                      "drop", str(dropout), "norm", batch_norm_options[batch_norm]])
    
    return run_name;

In [7]:
#Integrate WandB with training and validation process
def train():

    config_defaults = {
        "no_of_filters_1": 8,
        "no_of_filters_2": 16,
        "no_of_filters_3": 16,
        "no_of_filters_4": 32,
        "no_of_filters_5": 32,
        "augment_data": False,
        "dropout": 0.0,
        "batch_norm": False,
        "epochs": 10,
        "dense_size": 1000,
        "lr": 0.001
    }
    
    os.environ['WANDB_NOTEBOOK_NAME'] = 'A2_P1_2.ipynb'

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config
    wandb.run.name = setRunName([config.no_of_filters_1, config.no_of_filters_2, config.no_of_filters_3, config.no_of_filters_4, config.no_of_filters_5], config.augment_data, config.dropout, config.batch_norm)

    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data)
    model = createCNN(image_size, [config.no_of_filters_1, config.no_of_filters_2, config.no_of_filters_3, config.no_of_filters_4, config.no_of_filters_5], kernel_size, activation, config.batch_norm, config.dropout, config.dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")
    model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])

In [8]:
#Set up a sweep config
sweep_config = {
    "description": "Training and Checking the performance of CNN on validation data",
    "metric": {
    'name': 'val_accuracy',
    'goal': 'maximize'   
    },
    "method": "grid",
    "project": "DL_CS6910_Assignment2",
    "parameters": {
        "no_of_filters_1": {
            "values": [8, 16, 32]
        },
        "no_of_filters_2": {
            "values": [8, 16, 32]
        },
        "no_of_filters_3": {
            "values": [8, 16, 32]
        },
        "no_of_filters_4": {
            "values": [8, 16, 32]
        },
        "no_of_filters_5": {
            "values": [8, 16, 32]
        },
        "augment_data": {
            "values": [False, True]
        },
        "dropout": {
            "values": [0.0, 0.3]
        },
        "batch_norm": {
            "values": [False, True]
        },
        "epochs": {
            "values": [5]
        },
        "dense_size": {
            "values": [64, 100, 128]
        },
        "lr": {
            "values": [0.001, 0.01]
        }
    }
}

# creating the sweep
sweep_id = wandb.sweep(sweep_config, project="DL_CS6910_Assignment2")

Create sweep with ID: jzalq6jm
Sweep URL: https://wandb.ai/cs21m029_keyur_raval/DL_CS6910_Assignment2/sweeps/jzalq6jm


In [9]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: ryu5nrn1 with config:
wandb: 	augment_data: False
wandb: 	batch_norm: False
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	no_of_filters_1: 8
wandb: 	no_of_filters_2: 8
wandb: 	no_of_filters_3: 8
wandb: 	no_of_filters_4: 8
wandb: 	no_of_filters_5: 8
wandb: Currently logged in as: cs21m029_keyur_raval (use `wandb login --relogin` to force relogin)


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
180/180 [==============================] - 121s 620ms/step - loss: 2.2633 - accuracy: 0.1407 - val_loss: 2.1979 - val_accuracy: 0.1832
Epoch 2/5
180/180 [==============================] - 87s 486ms/step - loss: 2.1601 - accuracy: 0.2079 - val_loss: 2.1249 - val_accuracy: 0.2212
Epoch 3/5
180/180 [==============================] - 82s 456ms/step - loss: 2.1004 - accuracy: 0.2362 - val_loss: 2.1306 - val_accuracy: 0.2382
Epoch 4/5
180/180 [==============================] - 81s 449ms/step - loss: 2.0666 - accuracy: 0.2563 - val_loss: 2.0648 - val_accuracy: 0.2593
Epoch 5/5
180/180 [==============================] - 82s 455ms/step - loss: 2.0324 - accuracy: 0.2749 - val_loss: 2.0222 - val_accuracy: 0.2863



accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▄▅▆█
val_loss,█▅▅▃▁
accuracy,0.27489
best_epoch,4
best_val_loss,2.02222
epoch,4
loss,2.03239
val_accuracy,0.28629


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: raofbksq with config:
wandb: 	augment_data: False
wandb: 	batch_norm: False
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	no_of_filters_1: 8
wandb: 	no_of_filters_2: 8
wandb: 	no_of_filters_3: 8
wandb: 	no_of_filters_4: 8
wandb: 	no_of_filters_5: 16


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
140/180 [======================>.......] - ETA: 16s - loss: 2.2439 - accuracy: 0.1544

...

accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▆▄▂▁
val_accuracy,▁▄███
val_loss,█▆▂▂▁
accuracy,0.26767
best_epoch,4
best_val_loss,2.0553
epoch,4
loss,2.03726
val_accuracy,0.24525


wandb: Agent Starting Run: 1z50yaev with config:
wandb: 	augment_data: False
wandb: 	batch_norm: False
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	lr: 0.001
wandb: 	no_of_filters_1: 32
wandb: 	no_of_filters_2: 8
wandb: 	no_of_filters_3: 16
wandb: 	no_of_filters_4: 8
wandb: 	no_of_filters_5: 32


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
180/180 [==============================] - 72s 396ms/step - loss: 2.2566 - accuracy: 0.1419 - val_loss: 2.2053 - val_accuracy: 0.1812
Epoch 2/5
180/180 [==============================] - 69s 386ms/step - loss: 2.2009 - accuracy: 0.1829 - val_loss: 2.1976 - val_accuracy: 0.1902
Epoch 3/5
180/180 [==============================] - 74s 410ms/step - loss: 2.1740 - accuracy: 0.2010 - val_loss: 2.1995 - val_accuracy: 0.1852
Epoch 4/5
180/180 [==============================] - 75s 418ms/step - loss: 2.1517 - accuracy: 0.2058 - val_loss: 2.1736 - val_accuracy: 0.2172
Epoch 5/5
180/180 [==============================] - 77s 427ms/step - loss: 2.1365 - accuracy: 0.2089 - val_loss: 2.1598 - val_accuracy: 0.2002



accuracy,▁▅▇██
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▃▂█▅
val_loss,█▇▇▃▁
accuracy,0.20889
best_epoch,4
best_val_loss,2.15976
epoch,4
loss,2.13649
val_accuracy,0.2002


wandb: Ctrl + C detected. Stopping sweep.
